# Sequential Learning Project : sequential weighted scheme comparison
_Supervisor : Gilles Stoltz (http://stoltz.perso.math.cnrs.fr/enseignements.html)_

_Author : Hugo Vallet (https://uk.linkedin.com/in/hugovallet)_

_Course : Learning and sequential optimization, Data Science track, Paris-Saclay University_

## Intoduction
In this report, we will test multiple weighting schemes for sequential learning problems. As a reminder, the sequential learning setup is the following :
- We have a set of $N$ "experts", or predictors, $N \in \mathbb{N}$
- We get the data "online", meaning that we do not have access directly to a batch : the examples (data points in $\mathbb{R}^d$) are coming iteratively in time.
- At each step $t$, for a given data point $x_t$, the prediction is a weighted sum of the individual predictions of the  experts. 

The main challenge here is to find the best choice of weights at a time t, given the data (and the associated errors of the experts) seen in the past.

## Dataset
Here, I consider first a toy regression problem generated using the library Sklearn. This dataset is composed of 10000 iid obeservations in $\mathbb{R}^{15}$ following a Gaussian distribution.



In [184]:
#Import libraries
%matplolib inline

from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('hugovallet', '1po3tzu1j2')

#Generate the dataset
data,y = make_regression(n_samples = 10000, n_features = 15, n_informative = 15, n_targets = 1 , bias = 0, noise=0.1)

ERROR: Line magic function `%matplolib` not found.


## Choice of experts and methodology
The aim of the report is to study the behaviour of the Ridge-weighting and EWA algorithms on a given set of predictors for an incoming flow of data. Hence, we are not really interested in optimizing the choice of experts. Finally, these experts are fixed at the beginning of the problem : they are not intrinsectly modified during the online process, only their weights are updated.

With this setup, I created a set of random trees that I will use as experts. The trees are generated using a random-forest training approach :

For each tree :
- Select randomly a set of features on which the tree will be trained
- Construct the tree splitting iteratively on a random subset of this features. The split is chosen with a entropy-based criterion : the chosen split is the one maximizing the "information" gain.

I use half of my data for this initial training of the experts and I keep the remaining half for the study of the online weighting algorithms (Ridge and EWA). This approach seems valid in the sens that :
- The datapoints follow the same distribution. Thus, theorically, the experts learnt during the first phase are relevant for the sequential problem.
- Because of the random forest procedure (decribed above) the trees generated are really of variable efficiency. Some of them, which are based on very informative features, perform a lot better than others. Thus, finding a good way to average their predictions (and not taking just the basic average, as used in classical random forest prediction) is relevant.

### First phase : training of the experts (generating the trees)

In [185]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
reg = RandomForestRegressor(n_estimators=50, max_features='sqrt')
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
print "Adjusted R-squared score : ",r2_score(y_pred,y_test)

Adjusted R-squared score :  0.465491661664


When using random forests, it is always interesting to check the features which contribute the most is the model. Sklearn implementation proposes a keys-in-hand method to compute an individual score of the different features which is directly linked to the number of times the features where selected for a split (i.e. how informative they are).

In [190]:
feature_importances = reg.feature_importances_
trace = go.Bar(
         x=[("Feature"+str(i)) for i in range(len(feature_importances))],
         y=feature_importances
)
trace = [trace]
py.iplot(trace, filename='importances')

From this graph, it is easy to intuit that the trees which are not based on the features 5 and 14, for instance, will be less efficient than the other ones. To check that, let's plot the individual errors of our trees in the random forest.

In [189]:
from sklearn.metrics import mean_squared_error
trees = reg.estimators_
errors = []
for tree in trees :
    error = mean_squared_error(y_test,tree.predict(X_test))
    errors.append(error)
    
plt.figure()
plt.bar(np.arange(len(errors)),errors)
plt.show()
    

We can also plot the real value of series to predict and the prediction made by a uniform average of the predictions of the trees 

In [212]:
trace1 = go.Scatter(x = np.arange(len(y_test)),y = y_test,name = "Real value")
trace2 = go.Scatter(x = np.arange(len(y_pred)),y = y_pred, name = "Predicted value (uniform average of predictors)")
trace = [trace1, trace2]
py.iplot(trace, filename='prediction VS real value')

## Implementation of the sequential algorithms

Here are implemented to classical sequential agolrithms : EWA and Riddge.

### EWA (Exponentially Weighted Average)

In the EWA algorithm, the weights are updated respecting the following law :

$$ \forall j\in[1,N], w_{1,j} = \frac{1}{N} $$

$$ \forall(t,j)\in [2,T]\times[1,N],  w_{t,j} = \frac{1}{W}exp(-\eta\sum\limits_{s=1}^{t-1} l_{s,j}) $$

whith $W$ a normalization factor (to make the weights sum to 1) and $l_{s,j}$ the error (loss) observed for the predictor $j$ at the step $s$

In [192]:
def compute_EWA_weights(eta,weights_matrix,loss_matrix):
    """
    This method takes a weights and loss matrices as input (size t,N where t is the current number of iterations) 
    and returns the new row as well as the updated weights matrix for further computations.
    """
    weights = np.exp(-eta*np.sum(loss_matrix,axis=0))
    W = np.sum(weights)
    weights = np.expand_dims(weights/W,axis=0)
    
    return weights
    
def compute_losses(experts, X, y, loss = "least squares"):
    """
    This methods takes as input the loss function we want to use to measure the distance between the predictions of 
    the experts, for a given example X (vector of observations), and the real value y.
    The experts here are all sklearn regressors objects stocked in an array.
    """
    losses = []
    for expert in experts :
        pred = expert.predict(X)[0]
        if loss == "least squares" :
            obs_loss = (y-pred)**2
            losses.append(obs_loss)
    
    
    return losses

def weighted_prediction(experts,xt,weights):
    predictions = []
    for expert in experts :
        pred = expert.predict(xt)[0]
        predictions.append(pred)
    predictions = np.array(predictions)
    return np.sum(weights*predictions)
    
def draw_example(X,y):
    n_examples = X.shape[0]
    index = np.random.randint(low=0,high=n_examples)
    Xt = X[index,:].reshape(1, -1)
    yt = y[index]
    return Xt,yt

def compute_regret(weights_matrix,loss_matrix):
    """
    Compute the regret given the 2 matrices of interest.
    """
    #Compute the term on the left of the regret formula for a linear loss function 
    first_term = sum(sum(weights_matrix*loss_matrix))
    second_term = min(np.sum(loss_matrix,axis=0))
    return first_term - second_term

def compute_bound(N,eta,time,m,M):
    return np.log(N)/eta + eta*(M-m)**2/8*time   

Currently, we have a batch of data. To simulate the online approach, at each step t in the algorithm, we will draw uniformly an example on that batch.

In [194]:
def EWA(experts, X, y, T, eta):
    #Initialization
    N = len(experts)
    
    weights_matrix = np.zeros((T,N))
    weights_matrix[0,:] = 1/float(N)*np.ones((1,N))
    weights = weights_matrix[0,:]
    loss_matrix = np.zeros((T,N))
    #error vec 1 stocks the errors made by the aggregated model
    error_vec1=[]
    #error vec 2 stocks the error made by the random forest (for further performances comparisons)
    error_vec2=[]
    #The regrets at each step for our algorithm
    regrets = []
    
    #Start sequential learning
    for t in range(T):
        #Draw an example from the batch
        Xt,yt = draw_example(X,y)
        #Oberve the losses of the experts
        loss_matrix[t,:] = compute_losses(experts,Xt,yt,loss = "least squares")
        #Make a prediction
        y_pred = weighted_prediction(experts,Xt,weights)
        #Compute regret    
        regret = compute_regret(weights_matrix,loss_matrix)
        regrets.append(regret)
        #Update weights
        if t<T-1:
            weights_matrix[t+1,:] = compute_EWA_weights(eta,weights_matrix,loss_matrix)
            weights = weights_matrix[t+1,:]
            
        #Stock errors for plotting
        error1 = (yt-y_pred)**2
        error_vec1.append(error1)
        error2 = (yt-reg.predict(Xt)[0])**2
        error_vec2.append(error2)
    
    return loss_matrix, weights_matrix, error_vec1,error_vec2,regrets

In [195]:
X = X_test
y = y_test
T=100
experts = reg.estimators_
eta = 0.00000001
loss_matrix, weights_matrix, error_vec1,error_vec2,regrets = EWA(experts, X, y, T, eta)

In [200]:
plot_weights_3d(weights_matrix)

Because of the high number of experts, this plot is a bit messy, here is another visualization of the evolution of weights accross time :

In [210]:
plot_heatmap(weights_matrix)

Here it is clear that choosing a good $\eta$ is foundamental : a too large value will make the update rules too sensitive and thus block the weights on few experts, a too small value will restrain the algorithm's ability to adapt to the incoming flow of data and the prediction will be similar to a classical RF.

In the previous example I cheated : because I have a batch of data, I computed a first time the loss matrix to have an idea of $m$ and $M$ so that i could take an optimal fixed $\eta$.

### Variable step EWA

To have a better control on the regret, a classical approach is to make $\eta$ vary accross time. 
One possible implementation is to make the $\eta$ depending on the distribution chosen before. For that specific distribution we obtain the following bound : 

$$
\forall t>0, \eta_t = \frac{ln(N)}{\sum_{s=1}^{t-1} \delta_s} 
$$

## Appendix

Other functions I used in the report, notably visualization functions

In [199]:
def plot_weights_3d(matrix):
    """
    Note :
    This method is used to create 3D-graphs to plot the errors or weights accross time.
    - matrix : an T by N matrix containing the errors or weights of the N predictors for each iteration between 1 and T.
    """
    T,N = matrix.shape
    data = []
    for n in range(N):
        x = np.ones(T)
        y = np.arange(T)
        z = matrix[:T,n]

        trace = go.Scatter3d(
        x=n*x,
        y=y,
        z=z,
        mode='lines',
            marker=dict(
                size=12,
                line=dict(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.5   
                ),
                opacity=0.8,
                symbol = "circle",
            ),

        )
        data.append(trace)

    layout = go.Layout(
        title = "weights accross time",
        margin=dict(l=0, r=0, b=0, t=5),
        scene=go.Scene(
            xaxis=go.XAxis(title='Expert index'),
            yaxis=go.YAxis(title='Time'),
            zaxis=go.ZAxis(title='Weight')
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig, filename='simple-3d-scatter')


In [209]:
def plot_heatmap(weights_matrix):
    data = [
        go.Heatmap(
            z=weights_matrix,
            colorscale='Viridis'
        )
    ]

    layout = go.Layout(
        title='Evolution of the weights accross time',
        xaxis = dict(title = 'Expert index',),
        yaxis = dict(title = 'Number of iterations',)
    )
    figure = go.Figure(data=data, layout=layout)
    return py.iplot(figure, filename='weights-heatmap')